## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import random
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
nltk_data[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

 **Training and validation split**

In [4]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05, random_state =1234)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

3718
196
[[('This', 'DET'), ('year', 'NOUN'), (',', '.'), ('the', 'DET'), ('average', 'NOUN'), ('of', 'ADP'), ('daily', 'ADJ'), ('contracts', 'NOUN'), ('traded', 'VERB'), ('*', 'X'), ('totaled', 'VERB'), ('9,118', 'NUM'), (',', '.'), ('up', 'ADP'), ('from', 'ADP'), ('4,645', 'NUM'), ('a', 'DET'), ('year', 'NOUN'), ('earlier', 'ADJ'), ('and', 'CONJ'), ('from', 'ADP'), ('917', 'NUM'), ('in', 'ADP'), ('1984', 'NUM'), ('.', '.')], [('First', 'NOUN'), ('of', 'ADP'), ('America', 'NOUN'), (',', '.'), ('which', 'DET'), ('*T*-1', 'X'), ('now', 'ADV'), ('has', 'VERB'), ('45', 'NUM'), ('banks', 'NOUN'), ('and', 'CONJ'), ('$', '.'), ('12.5', 'NUM'), ('billion', 'NUM'), ('*U*', 'X'), ('in', 'ADP'), ('assets', 'NOUN'), (',', '.'), ('announced', 'VERB'), ('an', 'DET'), ('agreement', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('acquire', 'VERB'), ('the', 'DET'), ('Peoria', 'NOUN'), (',', '.'), ('Ill.', 'NOUN'), (',', '.'), ('bank', 'NOUN'), ('holding', 'VERB'), ('company', 'NOUN'), ('in', 'ADP'), ('January',

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95799

In [6]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['This',
 'year',
 ',',
 'the',
 'average',
 'of',
 'daily',
 'contracts',
 'traded',
 '*']

In [7]:
# vocabulary
V = set(tokens)
print(len(V))

12073


In [8]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print(len(T))
print(T)

12
{'PRON', 'CONJ', 'X', 'NOUN', 'DET', 'VERB', '.', 'ADV', 'PRT', 'NUM', 'ADP', 'ADJ'}


### Build the vanilla Viterbi based POS tagger

In [9]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    # count_tag = 0
    # count_w_given_tag = 0
    # for pair in train_bag:
    #     if pair[1]==tag:
    #         count_tag+=1
    #     if pair[0]==word:
    #         count_w_given_tag+=1
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [10]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [11]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [12]:
tags_matrix

array([[7.63650239e-03, 4.58190171e-03, 9.39289778e-02, 2.07331046e-01,
        9.16380342e-03, 4.87972498e-01, 4.04734649e-02, 3.28369625e-02,
        1.18365791e-02, 7.25467736e-03, 2.32913326e-02, 7.36922473e-02],
       [5.73694035e-02, 4.66417900e-04, 8.86194035e-03, 3.48880589e-01,
        1.18470147e-01, 1.58582091e-01, 3.31156701e-02, 5.59701510e-02,
        5.13059692e-03, 4.15111929e-02, 5.27052246e-02, 1.18936568e-01],
       [5.63291125e-02, 1.07594933e-02, 7.48417750e-02, 6.21835440e-02,
        5.41139245e-02, 2.04113930e-01, 1.62816450e-01, 2.61075944e-02,
        1.84651896e-01, 2.68987333e-03, 1.44936711e-01, 1.64556969e-02],
       [4.76866495e-03, 4.22627516e-02, 2.92308256e-02, 2.64897525e-01,
        1.32503370e-02, 1.46336138e-01, 2.39306912e-01, 1.71817560e-02,
        4.39736433e-02, 9.53732990e-03, 1.77023038e-01, 1.22310799e-02],
       [3.74894193e-03, 4.83734417e-04, 4.61966395e-02, 6.38650358e-01,
        5.44201257e-03, 3.95452902e-02, 1.77772399e-02, 1.26

In [13]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,PRON,CONJ,X,NOUN,DET,VERB,.,ADV,PRT,NUM,ADP,ADJ
PRON,0.007637,0.004582,0.093929,0.207331,0.009164,0.487972,0.040473,0.032837,0.011837,0.007255,0.023291,0.073692
CONJ,0.057369,0.000466,0.008862,0.348881,0.118470,0.158582,0.033116,0.055970,0.005131,0.041511,0.052705,0.118937
X,0.056329,0.010759,0.074842,0.062184,0.054114,0.204114,0.162816,0.026108,0.184652,0.002690,0.144937,0.016456
NOUN,0.004769,0.042263,0.029231,0.264898,0.013250,0.146336,0.239307,0.017182,0.043974,0.009537,0.177023,0.012231
DET,0.003749,0.000484,0.046197,0.638650,0.005442,0.039545,0.017777,0.012698,0.000242,0.022373,0.009191,0.203652
VERB,0.035321,0.005577,0.217816,0.110844,0.133617,0.167622,0.035167,0.083501,0.031216,0.022696,0.091402,0.065221
.,0.065768,0.057772,0.027314,0.223091,0.173226,0.088769,0.094070,0.051932,0.002336,0.080593,0.090386,0.044654
ADV,0.015400,0.006881,0.023263,0.031127,0.068480,0.344364,0.134666,0.081258,0.014744,0.031455,0.118611,0.129751
PRT,0.017915,0.002280,0.014007,0.247883,0.099674,0.402932,0.041694,0.009772,0.001954,0.056678,0.021173,0.084039
NUM,0.001486,0.013377,0.210464,0.354637,0.002973,0.018133,0.115933,0.002973,0.027051,0.184899,0.035672,0.032402


In [14]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in tqdm(enumerate(words), mininterval = 100):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_w_given_tag/ count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



Preparing test data

In [15]:
# # Let's test our Viterbi algorithm on a test dataset

# random.seed(1234)

# # choose random 5 sents
# rndom = [random.randint(1,len(test_set)) for x in range(5)]

# # list of sents
# test_run = [test_set[i] for i in rndom]
test_run = test_set

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('Ruth', 'NOUN'),
  ('K.', 'NOUN'),
  ('Nelson', 'NOUN'),
  ('Cullowhee', 'NOUN'),
  (',', '.'),
  ('N.C', 'NOUN'),
  ('.', '.')],
 [('Among', 'ADP'),
  ('other', 'ADJ'),
  ('Connecticut', 'NOUN'),
  ('banks', 'NOUN'),
  ('whose', 'PRON'),
  ('shares', 'NOUN'),
  ('*T*-121', 'X'),
  ('trade', 'VERB'),
  ('in', 'ADP'),
  ('the', 'DET'),
  ('OTC', 'NOUN'),
  ('market', 'NOUN'),
  (',', '.'),
  ('Society', 'NOUN'),
  ('for', 'ADP'),
  ('Savings', 'NOUN'),
  ('Bancorp', 'NOUN'),
  (',', '.'),
  ('based', 'VERB'),
  ('*', 'X'),
  ('in', 'ADP'),
  ('Hartford', 'NOUN'),
  (',', '.'),
  ('saw', 'VERB'),
  ('its', 'PRON'),
  ('stock', 'NOUN'),
  ('rise', 'VERB'),
  ('1', 'NUM'),
  ('3\\/4', 'NUM'),
  ('to', 'PRT'),
  ('18', 'NUM'),
  ('1\\/4', 'NUM'),
  ('.', '.')],
 [('Card', 'NOUN'),
  ('holders', 'NOUN'),
  ('who', 'PRON'),
  ('*T*-59', 'X'),
  ('receive', 'VERB'),
  ('the', 'DET'),
  ('letter', 'NOUN'),
  ('also', 'ADV'),
  ('are', 'VERB'),
  ('eligible', 'ADJ'),
  ('for', 'ADP'),
  ('a',

In [16]:
# tagging the test sentences,  takes upto 10 mins
tagged_seq = Viterbi(test_tagged_words)
# print(tagged_seq)

4877it [09:07,  8.91it/s]


#### Evaluating tagging accuracy

In [17]:
# accuracy for Vanilla Viterbi
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9060898093090014

**The Vanilla Viterbi yields an accuracy of 90.60%**

Lets analyse some incorrectly tagged words . This will help us create morphology based rules for unknown words.

In [18]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
## FORMAT => previuos word, predicted tag, actual tag
incorrect_tagged_cases

[[('.', '.'), (('Ruth', 'PRON'), ('Ruth', 'NOUN'))],
 [('Nelson', 'NOUN'), (('Cullowhee', 'PRON'), ('Cullowhee', 'NOUN'))],
 [('shares', 'NOUN'), (('*T*-121', 'PRON'), ('*T*-121', 'X'))],
 [('.', '.'), (('Card', 'PRON'), ('Card', 'NOUN'))],
 [('a', 'DET'), (('sweepstakes', 'PRON'), ('sweepstakes', 'NOUN'))],
 [('Maxwell', 'NOUN'), (('R.D.', 'PRON'), ('R.D.', 'NOUN'))],
 [('R.D.', 'NOUN'), (('Vos', 'PRON'), ('Vos', 'NOUN'))],
 [(',', '.'), (('N.Y', 'PRON'), ('N.Y', 'NOUN'))],
 [('The', 'DET'), (('Perch', 'PRON'), ('Perch', 'NOUN'))],
 [('and', 'CONJ'), (('Dolphin', 'PRON'), ('Dolphin', 'NOUN'))],
 [('producing', 'VERB'), (('early', 'ADV'), ('early', 'ADJ'))],
 [('the', 'DET'), (('Seahorse', 'PRON'), ('Seahorse', 'NOUN'))],
 [('and', 'CONJ'), (('Tarwhine', 'PRON'), ('Tarwhine', 'NOUN'))],
 [('be', 'VERB'), (('refunded', 'PRON'), ('refunded', 'VERB'))],
 [('newly', 'ADV'), (('fattened', 'PRON'), ('fattened', 'VERB'))],
 [('a', 'DET'), (('disembodied', 'PRON'), ('disembodied', 'ADJ'))],
 [

The vanilla Viterbu hueristic as randomly tagged the first tag i.e "NUM" to all unknown words. Lets modify the heuristic that assigns most frequent tag to unknown words.

### Solve the problem of unknown words

In [19]:
 ## extracting most frequent tag
 from collections import Counter
 tag_counts = Counter([tup[1] for tup in train_tagged_words])
 print(tag_counts.most_common(20))
 MOST_FREQ_POS_TAG = tag_counts.most_common(1)[0][0]
 print(MOST_FREQ_POS_TAG)

[('NOUN', 27471), ('VERB', 12910), ('.', 11130), ('ADP', 9387), ('DET', 8269), ('X', 6320), ('ADJ', 6063), ('NUM', 3364), ('PRT', 3070), ('ADV', 3052), ('PRON', 2619), ('CONJ', 2144)]
NOUN


We observe Noun is most frequent tag with an overall count of 27473 in training set.

##Model 1.1

In [20]:
####
## Viterbi Heuristic Modification 1.1
####
def Viterbi_lexicon(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in tqdm(enumerate(words), mininterval = 100):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            # if count_w_given_tag == 0:
                # print(words[key])
            emission_p = count_w_given_tag/ count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        # if word not in V:
        #   print(word)
        pmax = max(p)
        if pmax==0: ## means unknown word, OOV
          state_max = MOST_FREQ_POS_TAG
        else:
          # getting state for which probability is maximum
          state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [21]:
tagged_seq_lexicon = Viterbi_lexicon(test_tagged_words)
check = [i for i, j in zip(tagged_seq_lexicon, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_lexicon)
accuracy

4877it [09:28,  8.58it/s]


0.9374615542341603

**There is an increase in accuracy from 90.60% to 93.74% on assigning most frequent tag** . Lets create some more morphology based rules.

## Model 1.2

In [22]:
### prepare lexicon based rules
validation_oov = [word[1] for word in test_set if word[1] not in V]
validation_oov

[('K.', 'NOUN'),
 ('other', 'ADJ'),
 ('holders', 'NOUN'),
 ('R.D.', 'NOUN'),
 ('Perch', 'NOUN'),
 ('August', 'NOUN'),
 ('newly', 'ADV'),
 ('David', 'NOUN'),
 (',', '.'),
 ('oldest', 'ADJ'),
 ('a', 'DET'),
 ('recent', 'ADJ'),
 ('yield', 'NOUN'),
 ('cites', 'VERB'),
 ('Hammond', 'NOUN'),
 ('bonds', 'NOUN'),
 (',', '.'),
 ('an', 'DET'),
 ('Stung', 'VERB'),
 ('*', 'X'),
 ('appointment', 'NOUN'),
 ('1976', 'NUM'),
 ('buffet', 'NOUN'),
 ('the', 'DET'),
 ('forthcoming', 'ADJ'),
 ('provision', 'NOUN'),
 ('and', 'CONJ'),
 ('companies', 'NOUN'),
 ('also', 'ADV'),
 ('takeover', 'NOUN'),
 ('in', 'ADP'),
 ('the', 'DET'),
 ('they', 'PRON'),
 ('unlike', 'ADP'),
 ('is', 'VERB'),
 ('have', 'VERB'),
 ('training', 'NOUN'),
 ('Danforth', 'NOUN'),
 ('PRODUCTS', 'NOUN'),
 ('September', 'NOUN'),
 ('was', 'VERB'),
 (',', '.'),
 ('St.', 'NOUN'),
 ('an', 'DET'),
 ('of', 'ADP'),
 ('has', 'VERB'),
 ('Phelan', 'NOUN'),
 ('rose', 'VERB'),
 ('succeed', 'VERB'),
 ('Even', 'ADV'),
 ('most', 'ADJ'),
 ('said', 'VERB'),


In [23]:
## returns Tag based on word structure
def get_morphology_POS_tag(word):
  ## Or use nltk.RegexpTagger to do the following
  if word.endswith("ing") or word.endswith("ed"):
    return "VERB"
  elif word.endswith("ous"):
    return "ADJ"
  elif word.endswith("ly"):
    return "ADV"
  elif word.isnumeric():
    return "NUM"
  else:
    return MOST_FREQ_POS_TAG

In [24]:
### Model Modification 1.2 
def Viterbi_lexicon_morphology(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in tqdm(enumerate(words), mininterval = 100):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            emission_p = count_w_given_tag/ count_tag
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        if pmax == 0: ## OOV, 
          state_max = get_morphology_POS_tag(word)
        else:
          # getting state for which probability is maximum
          state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [25]:
tagged_seq_lexicon_morphology = Viterbi_lexicon_morphology(test_tagged_words)
check = [i for i, j in zip(tagged_seq_lexicon_morphology, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_lexicon_morphology)
accuracy

4877it [09:16,  8.76it/s]


0.947098626204634

**We observe further increase in accuracy to 94.71%**

Lets try different heuristic of taking most frequent tag after previous tag by considering only tranistion probailities in case of OOV.

## Model 2.1

In [31]:
### Model 2.1 Tansition based
def Viterbi_transition(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in tqdm(enumerate(words), mininterval = 100):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            # if count_w_given_tag == 0:
                # print(words[key])
            emission_p = count_w_given_tag/ count_tag
            state_probability = emission_p * transition_p    
            # if count_w_given_tag == 0:  #### doesn't work
            #   state_probability = transition_p
            p.append(state_probability)
            
        # if word not in V:
        #   print(word)
        pmax = max(p)
        if pmax==0: ## means unknown word, OOV
          statVar = state[-1] if len(state) > 0 else '.'
          state_max = tags_df.loc[statVar, :].idxmax(axis=1)
        else:
          # getting state for which probability is maximum
          state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [32]:
tagged_seq_transition = Viterbi_transition(test_tagged_words)
check = [i for i, j in zip(tagged_seq_transition, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_transition)
accuracy

4877it [08:43,  9.32it/s]


0.9384867746565512

**We again get an increase in accuracy from 90.60% in vanilla heuristic to 93.84% in the current model**.

Lets try considering previuos 2 tags to predict next tag in case of OOV words.

## Model 2.2

In [33]:
# compute tag given tag: tag3(t2,t1) given tag1 (t1), i.e. Transition Probability

def t3_given_t2t1(t3, t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t2_t1 = 0
    count_t3_t2_t1 = 0
    for index in range(len(tags)-2):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
            if tags[index+2] == t3:
              count_t3_t2_t1 += 1
    return (count_t3_t2_t1, count_t2_t1)

In [34]:
# creating t x t x t transition matrix of tags
# thus M(i, j, k) represents P(tk given ti, tj)

tags_matrix3d = np.zeros((len(T), len(T), len(T)), dtype='float32')
for i, t1 in tqdm(enumerate(list(T)),mininterval=10):
    for j, t2 in enumerate(list(T)): 
        for k, t3 in enumerate(list(T)): 
            occ, tot = t3_given_t2t1(t3, t2, t1)
            tags_matrix3d[i, j, k ] = occ/tot

12it [00:26,  2.22s/it]


In [35]:
tags_matrix3d

array([[[0.        , 0.        , 0.05      , ..., 0.        ,
         0.        , 0.05      ],
        [0.08333334, 0.        , 0.        , ..., 0.08333334,
         0.        , 0.        ],
        [0.00813008, 0.00813008, 0.00813008, ..., 0.        ,
         0.00406504, 0.00813008],
        ...,
        [0.        , 0.05263158, 0.        , ..., 0.10526316,
         0.        , 0.15789473],
        [0.06557377, 0.        , 0.19672132, ..., 0.06557377,
         0.03278688, 0.08196721],
        [0.        , 0.02590674, 0.00518135, ..., 0.01036269,
         0.04145078, 0.07253886]],

       [[0.00813008, 0.        , 0.07317073, ..., 0.01626016,
         0.        , 0.04878049],
        [0.        , 0.        , 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.15789473, 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.04494382,
         0.05617978, 0.02247191],
        [0.1

In [36]:
## tag to list index and reverse mappings
t2i = {}
i2t = {}
for i,t in enumerate(list(T)):
  t2i[t] = i
  i2t[i] = t

t2i

{'.': 6,
 'ADJ': 11,
 'ADP': 10,
 'ADV': 7,
 'CONJ': 1,
 'DET': 4,
 'NOUN': 3,
 'NUM': 9,
 'PRON': 0,
 'PRT': 8,
 'VERB': 5,
 'X': 2}

In [37]:
## verify
tags_matrix3d[t2i["NOUN"],t2i["VERB"]].argmax()

5

In [40]:
### Model 2 Tansition based  t3 given t2, t1
def Viterbi_transition_tri(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in tqdm(enumerate(words), mininterval = 100):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            count_w_given_tag, count_tag = word_given_tag(words[key], tag)
            # if count_w_given_tag == 0:
                # print(words[key])
            emission_p = count_w_given_tag/ count_tag
            state_probability = emission_p * transition_p    
            # if count_w_given_tag == 0:  #### doesn't work
            #   state_probability = transition_p
            p.append(state_probability)
            
        # if word not in V:
        #   print(word)
        pmax = max(p)
        if pmax==0 and key>0: ## means unknown word, OOV
          if key > 1:
            # print(words[key-2],words[key-1])
            # print(t2i[words[key-2]],t2i[words[key-1]])
            arr = tags_matrix3d[t2i[state[-2]],t2i[state[-1]],:]
            if arr.max()>0: ############## and arr.min()!=0   ## all occurences in training data given fare chance
              ind = arr.argmax()
              state_max = i2t[ind]
              # print(state_max)
            else:
              state_max = tags_df.loc[state[-1], :].idxmax(axis=1)
          else:
            state_max = tags_df.loc[state[-1], :].idxmax(axis=1)
        else:
          # getting state for which probability is maximum
          state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [41]:

tagged_seq_transition_tri = Viterbi_transition_tri(test_tagged_words)
check = [i for i, j in zip(tagged_seq_transition_tri, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_transition_tri)
accuracy

4877it [08:38,  9.40it/s]


0.9364363338117695

**We get similar accuracy . Now the accuray is 93.64%**

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

We'll compare the base viterbi model with the morphology based model (1.2) and transition based model (2.1) since they achieved the highest accuracy of 94.71% and 93.85%

In [42]:
## Testing sample set
sentence_test = '''Android is a mobile operating system developed by Google.
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
Twitter is an online news and social networking service on which users post and interact with messages known as tweets.
Before entering politics, Donald Trump was a domineering businessman and a television personality.
The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.
This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.
Show me the cheapest round trips from Dallas to Atlanta
I would like to see flights from Denver to Philadelphia.
Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.
NASA invited social media users to experience the launch of ICESAT-2 Satellite.'''
words = word_tokenize(sentence_test)

In [43]:
### Original heuristic
tagged_seq = Viterbi(words)

181it [00:18,  9.80it/s]


In [45]:
print(tagged_seq)

[('Android', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'PRON'), ('.', '.'), ('Android', 'PRON'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'PRON'), ('worldwide', 'PRON'), ('on', 'ADP'), ('smartphones', 'PRON'), ('since', 'ADP'), ('2011', 'PRON'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'PRON'), ('.', '.'), ('Google', 'PRON'), ('and', 'CONJ'), ('Twitter', 'PRON'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'PRON'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'PRON'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'PRON'), ("'s", 'VERB'), ('firehose', 'PRON'), ('.', '.'), ('Twitter', 'PRON'), ('is', 'VERB'), ('an', 'DET'), ('online', 'PRON'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'

### Comparing lexicon morphology based model (1.2) with base Vierbi Model

In [46]:

tagged_seq_lexicon_morphology = Viterbi_lexicon_morphology(words)


181it [00:19,  9.37it/s]


In [51]:
for i, word in enumerate(words):
  if tagged_seq_lexicon_morphology[i]!=tagged_seq[i]:
    prev_word = ""
    if i>0:
      prev_word = words[i-1]
    print(prev_word, word," => ",tagged_seq[i],"----->",tagged_seq_lexicon_morphology[i])

 Android  =>  ('Android', 'PRON') -----> ('Android', 'NOUN')
by Google  =>  ('Google', 'PRON') -----> ('Google', 'NOUN')
. Android  =>  ('Android', 'PRON') -----> ('Android', 'NOUN')
best-selling OS  =>  ('OS', 'PRON') -----> ('OS', 'NOUN')
OS worldwide  =>  ('worldwide', 'PRON') -----> ('worldwide', 'NOUN')
on smartphones  =>  ('smartphones', 'PRON') -----> ('smartphones', 'NOUN')
since 2011  =>  ('2011', 'PRON') -----> ('2011', 'NUM')
since 2013  =>  ('2013', 'PRON') -----> ('2013', 'NUM')
. Google  =>  ('Google', 'PRON') -----> ('Google', 'NOUN')
and Twitter  =>  ('Twitter', 'PRON') -----> ('Twitter', 'NOUN')
in 2015  =>  ('2015', 'PRON') -----> ('2015', 'NUM')
gave Google  =>  ('Google', 'PRON') -----> ('Google', 'NOUN')
to Twitter  =>  ('Twitter', 'PRON') -----> ('Twitter', 'NOUN')
Twitter 's  =>  ("'s", 'VERB') -----> ("'s", 'PRT')
's firehose  =>  ('firehose', 'PRON') -----> ('firehose', 'NOUN')
. Twitter  =>  ('Twitter', 'PRON') -----> ('Twitter', 'NOUN')
an online  =>  ('onlin

**Most of the OOVs are now tagged as NOUNS instead of PRONOUNS, while others as VERB, ADJ, ADV.  Specifically, the following words which were incorrectly tagged earlier are now correctly tagged:**



*   Google, Android, OS, smartphones, NASA have been correctly tagged from pronouun to noun.
*   2011, 2013, 2018 have been correctly tagged from PRONOUN to NUM
*   contested, invited, arriving have been correctly tagged from PRONOUN to VERB

### Comparing only transition based model (2.1) with base Vierbi Model

In [53]:

tagged_seq_transition = Viterbi_transition(words)


181it [00:21,  8.37it/s]


In [54]:
for i, word in enumerate(words):
  if tagged_seq_transition[i]!=tagged_seq[i]:
    prev_word = ""
    if i>0:
      prev_word = words[i-1]
    print(prev_word, word," => ",tagged_seq[i],"----->",tagged_seq_transition[i])

 Android  =>  ('Android', 'PRON') -----> ('Android', 'NOUN')
by Google  =>  ('Google', 'PRON') -----> ('Google', 'NOUN')
. Android  =>  ('Android', 'PRON') -----> ('Android', 'NOUN')
best-selling OS  =>  ('OS', 'PRON') -----> ('OS', 'NOUN')
OS worldwide  =>  ('worldwide', 'PRON') -----> ('worldwide', 'NOUN')
on smartphones  =>  ('smartphones', 'PRON') -----> ('smartphones', 'NOUN')
since 2011  =>  ('2011', 'PRON') -----> ('2011', 'NOUN')
since 2013  =>  ('2013', 'PRON') -----> ('2013', 'NOUN')
. Google  =>  ('Google', 'PRON') -----> ('Google', 'NOUN')
and Twitter  =>  ('Twitter', 'PRON') -----> ('Twitter', 'NOUN')
in 2015  =>  ('2015', 'PRON') -----> ('2015', 'NOUN')
gave Google  =>  ('Google', 'PRON') -----> ('Google', 'X')
to Twitter  =>  ('Twitter', 'PRON') -----> ('Twitter', 'VERB')
Twitter 's  =>  ("'s", 'VERB') -----> ("'s", 'PRT')
's firehose  =>  ('firehose', 'PRON') -----> ('firehose', 'VERB')
. Twitter  =>  ('Twitter', 'PRON') -----> ('Twitter', 'NOUN')
an online  =>  ('onlin

**As expected most of the OOVs are now tagged as NOUNS instead of PRONOUNS.  Specifically, the following words which were incorrectly tagged earlier are now correctly tagged:**



*   Google, Android, OS, FIFA smartphones, NASA, Satellite have been correctly tagged from pronouun to noun.

In [48]:
### Other models
### Taggers based on current word and previous 2 pos tags, But not for OOVs

from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, NgramTagger, SequentialBackoffTagger
  
tag0 = DefaultTagger(MOST_FREQ_POS_TAG)
tag1 = UnigramTagger(train_set, backoff = tag0) ## current the word
tag2 = BigramTagger(train_set, backoff = tag1)  ## previous tag and current the word
tag3 = TrigramTagger(train_set, backoff = tag2) ## previous 2 tags and current the word

# tag = backoff_tagger(train_set, 
#                      [UnigramTagger, BigramTagger, TrigramTagger], 
#                      backoff = backoff)
  
print("Training accuracy: ", tag3.evaluate(train_set))
print("Test accuracy: ", tag3.evaluate(test_set))

Training accuracy:  0.9909498011461497
Test accuracy:  0.9384867746565512


In [49]:
tag3.tag(["i","am"]), tag3.tag(["i","am","engineer"])

([('i', 'NOUN'), ('am', 'VERB')],
 [('i', 'NOUN'), ('am', 'VERB'), ('engineer', 'NOUN')])

In [50]:
# tags_df.loc["X", :].idxmax(axis=1)